In [5]:
import dspy
lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434')
dspy.configure(lm=lm)

# lm.history[-1].keys()  # access the last call to the LM, with all metadata



IndexError: list index out of range

In [4]:
len(lm.history) 

1

In [8]:
sentence = "the power of pettiness"  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment
# question = "How do the rhizome and lines of flight relate to Deleuze?"
# rag(context=search_wikipedia(question), question=question)

True

In [6]:
import pandas as pd
from typing import List, Union

class ColBERTv2:
    def __init__(self, csv_path: str):
        # Load the CSV into a DataFrame
        self.data = pd.read_csv(csv_path)
        # Ensure there is a column named "Ideas" to search
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> Union[List[str], List[dict]]:
        # Perform a simple search through the "Ideas" column
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():  # Basic substring matching
                results.append(idea)
                if len(results) >= k:
                    break

        if simplify:
            return results  # List of strings
        else:
            return [{"text": idea} for idea in results]  # List of dictionaries

# Usage Example
csv_path = "151_ideas_updated2.csv"  # Path to your CSV file

# Initialize the ColBERTv2 class with the CSV file
colbert = ColBERTv2(csv_path)

# Example query
query = "beauty"
results = colbert(query, k=3, simplify=True)

# Display results
for result in results:
    print("Result:", result)


Result: 1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) 
Result: 30) feminism - gitty up! Update 2-19-19 Feminism is important to me because I see half the population of the world suffering from a patriarchical prejudice that is stunting the growth of civilization. If that’s not enough to be a feminist, I don’t know what is. Prejudice is inherently dumb. Patriarchy is a biased archaic system. 

In [7]:
sentence = "maximize the beauty."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment

question = "How do you maximize the beauty?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning="The concept of maximizing beauty is complex and can be approached from different philosophical and psychological perspectives. From a hedonistic standpoint, beauty can be seen as a means to achieve pleasure and happiness. However, this perspective raises questions about the nature of beauty itself, as it is often tied to cultural and societal norms.\n\nIn the context of the feminine beauty ideal, maximizing beauty may involve conforming to certain standards of physical attractiveness, which can have negative psychological effects such as depression, eating disorders, and low self-esteem. This highlights the need for a more nuanced understanding of beauty that prioritizes individuality and self-acceptance over societal expectations.\n\nFrom an aesthetic perspective, beauty is often seen as a subjective experience that varies from person to person. Maximizing beauty may involve exploring and expressing one's own unique sense of style and aesthetics, rather than

In [13]:
import pandas as pd
import dspy
from typing import List

# Custom ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Simple substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")


# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    return results

# DSPy ChainOfThought setup
rag = dspy.ChainOfThought("context, question -> response")

# Example question
question = "what is the power of pettiness?"

# Pass context from the CSV search to the ChainOfThought model
response = rag(context=search_csv(question), question=question)

# Output response
print(response)


Prediction(
    reasoning='Pettiness can be defined as behavior that is petty, mean-spirited, or vindictive. It often involves taking small, insignificant actions to cause annoyance or harm to others.',
    response="The power of pettiness lies in its ability to create a toxic environment and undermine relationships. When individuals engage in petty behavior, it can escalate quickly, causing more harm than intended. Pettiness can also be a coping mechanism for people dealing with their own emotions, such as anger or frustration. However, it's essential to recognize that pettiness is not an effective way to resolve conflicts or address issues."
)


In [22]:
import pandas as pd
import dspy
from typing import List

# Enhanced ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Perform substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():  # Basic match
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    if not results:  # If no matches found, include a default context
        results = ["No exact matches found in the CSV, but here are some ideas to consider."]
    return results

# Correct DSPy ChainOfThought setup
rag = dspy.ChainOfThought("context, question -> response")

# Example question
question = "How do you maximize the beauty?"

# Pass context from the CSV search to the ChainOfThought model
context = search_csv(question)
response = rag(context=context, question=question)

# Output response
print("Context:", context)
print("Response:", response)


Context: ['No exact matches found in the CSV, but here are some ideas to consider.']
Response: Prediction(
    reasoning='The concept of maximizing beauty is subjective and can vary greatly depending on personal taste, cultural background, and individual experiences. Beauty can be found in nature, art, architecture, or even everyday moments.',
    response="While there's no one-size-fits-all approach to maximizing beauty, here are a few ideas to consider: appreciating the small things in life, finding joy in creative pursuits, cultivating meaningful relationships, and embracing diversity in all its forms. Ultimately, beauty is a personal and ever-evolving experience that can be discovered and nurtured with an open heart and mind."
)


In [28]:
import pandas as pd
import dspy
from typing import List, Dict

# Step 1: Create the basic CSV search class
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10) -> List[str]:
        results = []
        for idea in self.data["Ideas"].values:
            if query.lower() in str(idea).lower():
                results.append(idea)
                if len(results) >= k:
                    break
        return results

# Step 2: Initialize search and define search function
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

def search(query: str, k: int = 3) -> List[str]:
    return colbertv2_csv(query, k=k)

# Step 3: Define RAG module with correct signature format
class RAG(dspy.Module):
    def __init__(self, num_docs=3):
        super().__init__()
        self.num_docs = num_docs
        
        # Correct signature format: inputs -> outputs
        self.retrieve_and_respond = dspy.ChainOfThought(
            "context, question -> response"
        )

    def forward(self, question):
        # Retrieve context
        context = search(question, k=self.num_docs)
        
        # Generate response
        prediction = self.retrieve_and_respond(
            context=context,
            question=question
        )
        
        return prediction.response

# Step 4: Create simple training examples
def generate_training_examples() -> List[Dict]:
    # Start with just a few simple examples
    return [
        {"question": "How can we maximize beauty?"},
        {"question": "What are some creative ideas?"}
    ]

# Step 5: Set up optimization function
def optimize_rag():
    # Create training set
    trainset = generate_training_examples()
    
    # Initialize the optimizer with basic settings
    tp = dspy.MIPROv2(
        metric=dspy.evaluate.SemanticF1(decompositional=True),
        auto="medium",
        num_threads=4  # Reduced from 24 for testing
    )
    
    # Compile and optimize
    optimized_rag = tp.compile(
        RAG(),
        trainset=trainset,
        max_bootstrapped_demos=2,
        max_labeled_demos=2
    )
    
    return optimized_rag

# Step 6: Main function with error handling
def main():
    try:
        print("Optimizing RAG system...")
        optimized_rag = optimize_rag()
        
        question = "How do you maximize the beauty?"
        print(f"\nQuestion: {question}")
        
        response = optimized_rag(question)
        print(f"\nResponse: {response}")
        
    except Exception as e:
        
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

2025/01/14 17:37:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 1



Optimizing RAG system...


2025/01/14 17:38:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/14 17:38:44 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/14 17:38:44 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...
python(1463) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  0%|          | 0/1 [00:00<?, ?it/s]2025/01/14 17:38:44 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example {'question': 'How can we maximize beauty?'} with module = Predict(StringSignature(question, ground_truth, system_response -> reasoning, ground_truth_key_ideas, system_response_key_ideas, discussion, recall, precision
    instructions="Compare a system's response to the ground truth to compute recall and precision of key ideas.\nYou will first enumerate key ideas in each response, discuss their overlap, and then report recall and precision."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    ground_truth = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Ground Truth:', 'desc': '${ground_truth}'})
    system_response = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'System 

...

Average Metric: 0.00 / 0 (0%): 100%|██████████| 1/1 [00:00<00:00, 1825.99it/s]


2025/01/14 17:40:52 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 1 (0.0%)
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 18', 'Predictor 0: Few-Shot Set 8'].
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====
2025/01/14 17:40:52 ERROR dspy.utils.parallelizer: Error processing item {'question': 'What are some creative ideas?'}: 'dict' object has no attribute 'inputs'. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 0 (0%): 100%|██████████| 1/1 [00:00<00:00, 367.24it/s]

2025/01/14 17:40:52 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 1 (0.0%)
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 18'].
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:40:52 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 0.0!




Question: How do you maximize the beauty?

Response: There is no one-size-fits-all answer to maximizing beauty, as it varies greatly from person to person. However, some common ways to appreciate and enhance beauty include appreciating nature, art, music, and cultural heritage, while also taking care of oneself and others.


In [30]:
import pandas as pd
import dspy
from typing import List, Dict
import re

# Step 1: CSV handling class
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path, quoting=1)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")
        
        self.processed_ideas = self._preprocess_ideas()
    
    def _preprocess_ideas(self) -> List[Dict]:
        processed = []
        for idea in self.data["Ideas"].values:
            text = re.sub(r'^\d+\)\s*', '', str(idea))
            parts = text.split('-', 1)
            if len(parts) > 1:
                title = parts[0].strip()
                content = parts[1].strip()
            else:
                title = ""
                content = parts[0].strip()
            
            processed.append({
                "title": title,
                "content": content,
                "full_text": text
            })
        return processed

    def __call__(self, query: str, k: int = 10) -> List[str]:
        results = []
        query = query.lower()
        
        for idea in self.processed_ideas:
            if (query in idea["title"].lower() or 
                query in idea["content"].lower()):
                results.append(idea["full_text"])
                if len(results) >= k:
                    break
        return results

# Step 2: Initialize search
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

def search(query: str, k: int = 3) -> List[str]:
    results = colbertv2_csv(query, k=k)
    return [f"Relevant idea: {result}" for result in results]

# Step 3: RAG module with fixed signature
class RAG(dspy.Module):
    def __init__(self, num_docs=3):
        super().__init__()
        self.num_docs = num_docs
        # Simplified signature without additional instructions
        self.retrieve_and_respond = dspy.ChainOfThought("context, question -> response")

    def forward(self, question):
        context = search(question, k=self.num_docs)
        prediction = self.retrieve_and_respond(
            context="\n".join(context),
            question=question
        )
        return prediction.response

# Step 4: Training examples
def generate_training_examples() -> List[Dict]:
    return [
        {
            "question": "How can we maximize beauty?",
            "response": "Based on the philosophical ideas presented, maximizing beauty involves fully channeling the beauty within and actively seeking beauty in every situation. This includes disciplining our senses to focus on beauty in our surroundings, such as appreciating flowers and birds. It's considered a fundamental virtue that requires conscious implementation and can be enhanced by questioning what makes each moment beautiful."
        },
        {
            "question": "What is full expression?",
            "response": "Full expression is about understanding oneself when comfortable and learning to channel authentic self-expression. It recognizes that people face barriers like shyness and distraction that can prevent them from fully expressing themselves. The concept emphasizes the effort required to overcome these obstacles and achieve genuine self-expression."
        },
        {
            "question": "What does expect rising mean?",
            "response": "Expect Rising refers to the concept of constantly increasing expectations, similar to the saying 'give them an inch, they'll take a mile.' It relates to the law of diminishing returns, where adding more of something eventually yields lower incremental benefits. This principle applies to various aspects of life, from material goods to experiences."
        }
    ]

# Step 5: Optimization
def optimize_rag():
    trainset = generate_training_examples()
    
    tp = dspy.MIPROv2(
        metric=dspy.evaluate.SemanticF1(decompositional=True),
        auto="medium",
        num_threads=4
    )
    
    optimized_rag = tp.compile(
        RAG(),
        trainset=trainset,
        max_bootstrapped_demos=3,
        max_labeled_demos=3
    )
    
    return optimized_rag

# Step 6: Main function
def main():
    try:
        print("Optimizing RAG system...")
        optimized_rag = optimize_rag()
        
        test_questions = [
            "How do you maximize the beauty?",
            "What is the power of pettiness?",
            "How can we achieve full expression?"
        ]
        
        for question in test_questions:
            print(f"\nQuestion: {question}")
            response = optimized_rag(question)
            print(f"Response: {response}\n")
            print("-" * 50)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025/01/14 17:47:32 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 2



Optimizing RAG system...


2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  0%|          | 0/1 [00:00<?, ?it/s]2025/01/14 17:47:37 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example {'question': 'How can we maximize beauty?', 'response': "Based on the philosophical ideas presented, maximizing beauty involves fully channeling the beauty within and actively seeking beauty in every situation. This includes disciplining our senses to focus on beauty in our surroundings, such as appreciating flowers and birds. It's considered a fundamental virtue that requires conscious implementation and can be enhanced by questioning what makes each moment beautiful."} with module = Predict(StringSignature(question, ground_truth, system_response -> reasoning, ground_truth_key_ideas, system_response_key_ideas, discussion, recall, precision
    instructions="Compare a system's response to the ground truth to compute recall and precision of key ideas.\nYou will first enumerate key ideas in each response, discuss their overlap, and then report recall and precisio

...


Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 4415.06it/s]

2025/01/14 17:49:12 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 18', 'Predictor 0: Few-Shot Set 8'].
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====
2025/01/14 17:49:12 ERROR dspy.utils.parallelizer: Error processing item {'question': 'What does expect rising mean?', 'response': "Expect Rising refers to the concept of constantly increasing expectations, similar to the saying 'give them an inch, they'll take a mile.' It relates to the law of diminishin


Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 1926.64it/s]

2025/01/14 17:49:12 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 18'].
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 0.0!




Question: How do you maximize the beauty?
Response: The concept of beauty is subjective and can be influenced by various elements, including historical context, cultural norms, and individual experiences. Maximizing the beauty in a given situation or work of art may require a deep understanding of its intended message, audience, and the emotions it aims to evoke.

--------------------------------------------------

Question: What is the power of pettiness?
Response: Pettiness can be a powerful tool for social manipulation, allowing individuals to exert influence over others without directly confronting them. However, it can also lead to feelings of resentment and frustration among those on the receiving end, ultimately undermining any potential power or control that pettiness may offer.

--------------------------------------------------

Question: How can we achieve full expression?
Response: To achieve full expression in a given statement or condition, it is essential to carefully 

In [25]:
# failed advanced rag
import pandas as pd
import dspy
from typing import List

# Custom ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Simple substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    return results

# DSPy ChainOfThought setup with explicit CSV dependency
rag = dspy.ChainOfThought(
    signature=(
        "context, question -> response\n"
        "context: A list of ideas directly retrieved from the CSV based on the user's query.\n"
        "Only use these ideas to answer the question. If no matches are found, respond with\n"
        "'No relevant information available in the CSV.'\n"
        "question: The user's query.\n"
        "response: A response strictly based on the ideas provided in the context."
    )
)

# Example question
question = "How do you maximize the beauty?"

# Pass context from the CSV search to the ChainOfThought model
response = rag(context=search_csv(question), question=question)

# Output response
print(response)

SyntaxError: unterminated string literal (detected at line 2) (<unknown>, line 2)

In [ ]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")